In [48]:
# Actualizamos los repositorios
!apt-get update -qq

# Instalamos Spark para Python
!pip install pyspark

# Instalamos Java 8 (versión ligera sin GUI)
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

# Configuramos JAVA_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Seleccionamos Java 8 (si es necesario)
!echo 2 | update-alternatives --config java

# Validamos instalación de Java
!java -version

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
  0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
* 2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: openjdk version "1.8.0_432"
OpenJDK Runtime Environment (build 1.8.0_432-8u432-ga~us1-0ubuntu2~22.04-ga)
OpenJDK 64-Bit Server VM (build 25.432-bga, mixed mode)


In [136]:
from pyspark.sql import SparkSession
import os

# Crear siempre una nueva SparkSession
# Detenemos cualquier SparkSession o SparkContext existente para asegurarnos
# que el nuevo contexto esté limpio.
try:
    spark = SparkSession.builder.getOrCreate()
    spark.stop()
except:
    pass
finally:
  spark = SparkSession.builder.appName("CABD-TP2").getOrCreate()

# Definir rutas
root_path = '/content/'
input_path = root_path + "/input/jugadores.txt"
output_path = root_path + "/output/output1.txt"
# Crear el directorio de salida si no existe
if not os.path.exists(os.path.dirname(output_path)):
    os.makedirs(os.path.dirname(output_path))
# Eliminar el archivo de salida si ya existe
if os.path.exists(output_path):
    os.remove(output_path)

# Estructura del archivo:
# ID_Jugador, ID_Misión, ID_Héroe_1, ID_Héroe_2, ID_Héroe_3, ID_Héroe_4,
# ID_Héroe_5, Puntaje obtenido, Tiempo de la misión en segundos

# Leer, formatear datos y transformar a RDD KeyPair
jugadores = spark.sparkContext.textFile(input_path).map(lambda line: line.split("\t"))
misiones = jugadores.map(lambda fields: (int(fields[1]), 1))


mision_mas_jugada = misiones.reduceByKey(lambda row1, row2: row1 + row2) \
                    .takeOrdered(1, key=lambda x: -x[1])[0]

# Mostrar y guardar resultado
print(f"Misión más jugada: la {mision_mas_jugada[0]}, con {mision_mas_jugada[1]} veces en total.")
with open(output_path, "w") as file:
    file.write(f"Misión más jugada: la {mision_mas_jugada[0]}, con {mision_mas_jugada[1]} veces en total.\n")

Misión más jugada: la 1, con 22 veces en total.


In [137]:
from pyspark.sql import SparkSession, Row, Window, functions as F
import os

# Crear siempre una nueva SparkSession
# Detenemos cualquier SparkSession o SparkContext existente para asegurarnos
# que el nuevo contexto esté limpio.
try:
    spark = SparkSession.builder.getOrCreate()
    spark.stop()
except:
    pass
finally:
  spark = SparkSession.builder.appName("CABD-TP2").getOrCreate()

# Definir rutas
root_path = '/content'
input_path = root_path + "/input/jugadores.txt"
output_dir = root_path + "/output"
final_output_path = output_dir + "/output2_final.txt"
# Crear el directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# Eliminar el archivo final si ya existe
if os.path.exists(final_output_path):
    os.remove(final_output_path)

# Leer archivo como RDD
jugadores_rdd = spark.sparkContext.textFile(input_path)

# Aplanar los héroes por misión
misiones_y_heroes_rdd = jugadores_rdd.map(lambda line: line.split("\t")) \
    .flatMap(lambda fields: [(int(fields[1]), int(fields[i])) for i in range(2, 7)])

# Convertir a DataFrame
misiones_y_heroes_df = spark.createDataFrame(misiones_y_heroes_rdd, ["ID_Mision", "ID_Heroe"])

# Contar apariciones de héroes por misión
conteos = misiones_y_heroes_df.groupBy("ID_Mision", "ID_Heroe").count()

# Usar ventana para encontrar el máximo conteo por misión
windowPorMision = Window.partitionBy("ID_Mision")
max_conteo = conteos.withColumn("max_conteo", F.max("count").over(windowPorMision)) \
                    .filter(F.col("count") == F.col("max_conteo")) \
                    .drop("max_conteo") \
                    .orderBy("ID_Mision", "ID_Heroe")

# Agrupar héroes empatados por misión
resultado = max_conteo.groupBy("ID_Mision", "count") \
                      .agg(F.collect_list("ID_Heroe").alias("Heroes")) \
                      .orderBy("ID_Mision")

# Formatear el resultado por fila
def formatear_resultado(row):
    mision = row["ID_Mision"]
    heroes = row["Heroes"]
    count = row["count"]

    if len(heroes) == 1:
        heroes_str = f"héroe {heroes[0]}"
    else:
        heroes_str = "héroes " + " y ".join(map(str, heroes))

    return f"Misión {mision} ==> {heroes_str} ({count} veces)\n"

# Guardar distribuidamente
def guardar_en_archivo(partition):
    # Modo append para no sobreescribir lo que otras particiones escriban
    with open(final_output_path, "a") as file:
        for row in partition:
            file.write(formatear_resultado(row))

# Pasar a RDD para usar foreachPartition (ejecución del DAG) y guardar distribuidamente
resultado.rdd.foreachPartition(guardar_en_archivo)

print(f"Consulta 2 guardada en: {final_output_path}")

Consulta 2 guardada en: /content/output/output2_final.txt


In [138]:
from pyspark.sql import SparkSession, functions as F
import os

# Garantizar una SparkSession limpia
def iniciar_spark(app_name):
    try:
        # Detener cualquier sesión previa
        spark = SparkSession.builder.getOrCreate()
        spark.stop()
    except Exception as e:
        print(f"No se pudo detener la sesión previa: {e}")
    finally:
        # Crear una nueva sesión
        return SparkSession.builder.appName(app_name).getOrCreate()

# Crear la SparkSession
spark = iniciar_spark("CABD-TP3")

# Parámetro H: cantidad mínima de héroes distintos
H = 20

# Definir rutas
root_path = '/content'
input_path = root_path + "/input/jugadores.txt"
output_dir = root_path + "/output"
final_output_path = output_dir + "/output3_final.txt"
# Crear el directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# Eliminar el archivo final si ya existe
if os.path.exists(final_output_path):
    os.remove(final_output_path)

# Leer el archivo como DataFrame
jugadores_df = spark.read.option("delimiter", "\t").csv(input_path, inferSchema=True) \
                         .toDF("ID_Jugador", "ID_Mision", "ID_Heroe_1", "ID_Heroe_2",
                               "ID_Heroe_3", "ID_Heroe_4", "ID_Heroe_5", "Puntaje", "Tiempo")

# Aplanar los héroes por jugador, seleccionando las columnas de interés
jugador_heroes_df = jugadores_df.select(
    "ID_Jugador", F.explode(F.array("ID_Heroe_1", "ID_Heroe_2", "ID_Heroe_3",
                                    "ID_Heroe_4", "ID_Heroe_5")).alias("ID_Heroe")
)

# Contar héroes distintos por jugador y filtrarlos por parámetro H
jugadores_filtrados = jugador_heroes_df.groupBy("ID_Jugador") \
                                      .agg(F.countDistinct("ID_Heroe") \
                                      .alias("Cantidad_Heroes")) \
                                      .filter(F.col("Cantidad_Heroes") > H)

# Formatear y guardar el resultado
def formatear_resultado(partition):
    is_first_partition = False  # Bandera para escribir el encabezado
    if not os.path.exists(final_output_path):
        is_first_partition = True
    with open(final_output_path, "a") as file:
        if is_first_partition:
            file.write(f"Con H={H}\n")
        for row in partition:
            jugador = row["ID_Jugador"]
            cant_heroes = row["Cantidad_Heroes"]
            file.write(f"Jugador {jugador} con {cant_heroes} héroes distintos\n")

# Formatear y guardar en un archivo de texto distribuidamente (ejecución del DAG)
jugadores_filtrados.rdd.foreachPartition(formatear_resultado)

print(f"Consulta 3 guardada en: {final_output_path}")

Consulta 3 guardada en: /content/output/output3_final.txt


In [143]:
from pyspark.sql import SparkSession, functions as F
import os

# Garantizar una SparkSession limpia
def iniciar_spark(app_name):
    try:
        # Detener cualquier sesión previa
        spark = SparkSession.builder.getOrCreate()
        spark.stop()
    except Exception as e:
        print(f"No se pudo detener la sesión previa: {e}")
    finally:
        # Crear una nueva sesión
        return SparkSession.builder.appName(app_name).getOrCreate()

# Crear la SparkSession
spark = iniciar_spark("CABD-TP3")

# Parámetros de la consulta
N = 1  # Cantidad mínima de participaciones
P = 5000  # Puntaje mínimo total

# Definir rutas
root_path = '/content'
input_path = root_path + "/input/jugadores.txt"
output_dir = root_path + "/output"
final_output_path = output_dir + "/output4_final.txt"
# Crear el directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# Eliminar el archivo final si ya existe
if os.path.exists(final_output_path):
    os.remove(final_output_path)

# Leer el archivo como DataFrame
jugadores_df = spark.read.option("delimiter", "\t").csv(input_path, inferSchema=True) \
                         .toDF("ID_Jugador", "ID_Mision", "ID_Heroe_1", "ID_Heroe_2",
                               "ID_Heroe_3", "ID_Heroe_4", "ID_Heroe_5", "Puntaje", "Tiempo")

conteo_por_misiones = jugadores_df.select("ID_Mision", "ID_Jugador") \
                              .groupBy("ID_Mision") \
                                .agg(F.countDistinct("ID_Jugador") \
                                     .alias("Jugadores_Distintos"))

# Cachear para prevenir ejecutarse el mismo DAG más de una vez
conteo_por_misiones.cache()

maximo_conteo_por_misiones = conteo_por_misiones.agg(F.max("Jugadores_Distintos") \
                                   .alias("max_distintos")).first()["max_distintos"]

mision_mas_jugada = conteo_por_misiones.filter(
                                        F.col("Jugadores_Distintos")
                                        == maximo_conteo_por_misiones) \
                                        .agg(F.min("ID_Mision").alias("ID_Mision")) \
                                        .first()["ID_Mision"]

# Jugadores que participaron en la misión más jugada
jugadores_en_mision_mas_jugada = jugadores_df.filter(
                              F.col("ID_Mision")
                              == mision_mas_jugada)

# Al menos N veces con puntaje > P
cumplen = jugadores_en_mision_mas_jugada \
                .groupBy("ID_Jugador") \
                .agg(
                F.sum("Puntaje").alias("Puntaje_Total"), \
                F.count("ID_Jugador").alias("Cantidad_Participaciones")
                ) \
                .filter((F.col("Cantidad_Participaciones") > N)  \
                & (F.col("Puntaje_Total") > P))

# Tiempo total misión más jugada
tiempo_mision_mas_jugada = jugadores_en_mision_mas_jugada \
                           .groupBy("ID_Mision") \
                           .agg(
                               F.sum("Tiempo").alias("Tiempo_Total")
                           ).first()["Tiempo_Total"]

# Guardar encabezado y resultados
def guardar_resultados(partition):
    is_first_partition = not os.path.exists(final_output_path)
    with open(final_output_path, "a") as file:
        if is_first_partition:
            file.write(f"Con N={N} y P={P}\n")
            file.write(f"La misión más jugada es la {mision_mas_jugada} con {maximo_conteo_por_misiones} jugadores distintos y {tiempo_mision_mas_jugada} segundos de juego.\n")
        for row in partition:
            file.write(f"Jugador {row['ID_Jugador']}, participó {row['Cantidad_Participaciones']} veces con {row['Puntaje_Total']} en total\n")

# Guardar resultados distribuidamente (se ejecuta lo necesario del DAG)
cumplen.rdd.foreachPartition(guardar_resultados)

print(f"Consulta 4 guardada en: {final_output_path}")

Consulta 4 guardada en: /content/output/output4_final.txt


In [153]:
from pyspark.sql import SparkSession, functions as F
import os

# Garantizar una SparkSession limpia
def iniciar_spark(app_name):
    try:
        # Detener cualquier sesión previa
        spark = SparkSession.builder.getOrCreate()
        spark.stop()
    except Exception as e:
        print(f"No se pudo detener la sesión previa: {e}")
    finally:
        # Crear una nueva sesión
        return SparkSession.builder.appName(app_name).getOrCreate()

# Crear la SparkSession
spark = iniciar_spark("CABD-TP3")

# Parámetros de la consulta
N = 1  # Cantidad mínima de participaciones
P = 5000  # Puntaje mínimo total

# Definir rutas
root_path = '/content'
input_path = root_path + "/input/jugadores.txt"
output_dir = root_path + "/output"
final_output_path = output_dir + "/output5_final.txt"
# Crear el directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# Eliminar el archivo final si ya existe
if os.path.exists(final_output_path):
    os.remove(final_output_path)

# Leer el archivo como DataFrame
jugadores_df = spark.read.option("delimiter", "\t").csv(input_path, inferSchema=True) \
                         .toDF("ID_Jugador", "ID_Mision", "ID_Heroe_1", "ID_Heroe_2",
                               "ID_Heroe_3", "ID_Heroe_4", "ID_Heroe_5", "Puntaje", "Tiempo")

# Aplanar los héroes por jugador, seleccionando las columnas de interés
heroes_jugador_df = jugadores_df.select(
    "ID_Jugador", F.explode(F.array("ID_Heroe_1", "ID_Heroe_2", "ID_Heroe_3",
                                    "ID_Heroe_4", "ID_Heroe_5")).alias("ID_Heroe")
)

# Contar jugadores distintos por héroe
jugadores_distintos = heroes_jugador_df.groupBy("ID_Heroe") \
                                      .agg(F.countDistinct("ID_Jugador") \
                                      .alias("Cantidad_Jugadores"))


# Aplanar los héroes por misión, seleccionando las columnas de interés
heroes_mision_df = jugadores_df.select(
    "ID_Mision", F.explode(F.array("ID_Heroe_1", "ID_Heroe_2", "ID_Heroe_3",
                                    "ID_Heroe_4", "ID_Heroe_5")).alias("ID_Heroe")
)

# Contar misiones distintas por héroe
misiones_distintas = heroes_mision_df.groupBy("ID_Heroe") \
                                      .agg(F.countDistinct("ID_Mision") \
                                      .alias("Cantidad_Misiones"))

indice_p_df = jugadores_distintos.join(misiones_distintas, "ID_Heroe") \
              .withColumn("Indice_P", F.least(F.col("Cantidad_Jugadores"), F.col("Cantidad_Misiones"))) \
              .orderBy("ID_Heroe").drop("Cantidad_Jugadores", "Cantidad_Misiones")

# Formatear el resultado por fila
def formatear_resultado(row):
    heroe = row["ID_Heroe"]
    indice_p = row["Indice_P"]
    return f"Héroe {heroe}: Índice P = {indice_p}\n"

# Guardar distribuidamente
def guardar_en_archivo(partition):
    # Modo append para no sobreescribir lo que otras particiones escriban
    with open(final_output_path, "a") as file:
        for row in partition:
            file.write(formatear_resultado(row))

# Usar foreachPartition (ejecución del DAG) y guardar distribuidamente
indice_p_df.rdd.foreachPartition(guardar_en_archivo)

print(f"Consulta 5 guardada en: {final_output_path}")

Consulta 5 guardada en: /content/output/output5_final.txt
